# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vorgashor,67.5833,63.9500,-2.88,92,100,5.98,RU,1732469868
1,1,khategaon,22.6000,76.9167,17.40,47,9,2.09,IN,1732469869
2,2,albany,42.6001,-73.9662,6.99,60,8,0.89,US,1732469728
3,3,adamstown,-25.0660,-130.1015,22.16,66,53,6.55,PN,1732469872
4,4,terbanggi besar,-4.8667,105.2333,25.21,91,100,1.24,ID,1732469873


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = True,
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols=["City", "Country", "Max Temp"]  # Add 'city' to the tooltip
)

# Display the map plot
map_plot

# Display the map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Max Temp)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

# Ideal conditions
ideal_temp_min = 20
ideal_temp_max = 40
cloudiness = 50
max_lat = 30
min_lat = -60

# Filter the DataFrame
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] >= ideal_temp_min) &
    (city_data_df['Max Temp'] <= ideal_temp_max) &
    (city_data_df['Cloudiness'] <= cloudiness) &
    (city_data_df['Lat'] <= max_lat) &
    (city_data_df['Lat'] >= min_lat)
    ].reset_index(drop=True)

# Display the filtered DataFrame
ideal_weather_df

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,6,port mathurin,-19.6833,63.4167,24.78,72,36,7.88,MU,1732469875
1,7,tocopilla,-22.0920,-70.1979,21.59,68,48,5.13,CL,1732469876
2,8,tofol,5.3248,163.0078,29.26,73,32,7.09,FM,1732469877
3,13,bardai,11.6500,7.2333,22.46,21,5,4.05,NG,1732469883
4,20,san patricio,28.0170,-97.5169,27.97,46,5,7.87,US,1732469891
...,...,...,...,...,...,...,...,...,...,...
118,561,nova sintra,14.8667,-24.7167,25.19,74,2,6.54,CV,1732470531
119,578,rongelap,11.1635,166.8941,28.22,79,36,9.49,MH,1732470550
120,579,karachi,24.9056,67.0822,22.90,78,0,1.54,PK,1732470469
121,580,cidade velha,14.9167,-23.6167,30.26,35,0,5.66,CV,1732470553


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = None

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,6,port mathurin,-19.6833,63.4167,24.78,72,36,7.88,MU,1732469875,None
1,7,tocopilla,-22.0920,-70.1979,21.59,68,48,5.13,CL,1732469876,None
2,8,tofol,5.3248,163.0078,29.26,73,32,7.09,FM,1732469877,None
3,13,bardai,11.6500,7.2333,22.46,21,5,4.05,NG,1732469883,None
4,20,san patricio,28.0170,-97.5169,27.97,46,5,7.87,US,1732469891,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
hotel_rows = []

# Set the parameters for the type of place
categories = "accommodation.hotel"
limit = 20
radius = 10000

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    city = row["City"]
    country = row["Country"]
    latitude = row["Lat"]
    longitude = row["Lng"]
    humidity = row["Humidity"]

    # Set the parameters for the type of search
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    
    params = {
        "categories":categories,
        "limit":limit,
        "filter":filters,
        "bias":bias,
        "apiKey":geoapify_key
}

    # Print a message to follow up the hotel search
    pprint(f"Starting hotel search for {city}, {country}")

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Construct the full query URL with parameters
    query_url = f"{base_url}?{requests.compat.urlencode(params)}"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
        
    # Convert the API response to JSON format
    response = response.json()

    try:
        hotel_df.loc[index, "Airport Name"] = response["features"][0]["properties"]["name"]
        hotel_df.loc[index, "IATA Name"] = response["features"][0]["properties"]["datasource"]["raw"]["iata"]
        hotel_df.loc[index, "Airport Address"] = response["features"][0]["properties"]["address_line2"]
        hotel_df.loc[index, "Distance"] = response["features"][0]["properties"]["distance"]
        hotel_df.loc[index, "Website"] = response["features"][0]["properties"]["datasource"]["raw"]["website"]
    except KeyError as e:
        # If no airport is found, log the error.
        print(f"{e.args[0]} not found for {cities_pd.loc[index, 'Airport Name']}")
        
    # Log the search results
    print(f"nearest airport from {city}: {cities_pd.loc[index, 'Airport Name']}")

# Display sample data
cities_pd

'Starting hotel search for port mathurin, MU'


NameError: name 'cities_pd' is not defined

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE